# MRIO eval with Python for MSc theses
Overview of how basic MRIO analysis works
The numbers in this tutorial are adapted to EXIOBASE2 (163 products/industries in 48 regions). Real data are not presented here, only the principles.

In [12]:
import numpy as np
import xlrd, xlwt
import scipy.io



In [26]:
Filestring_Matlab_in      = 'C:\\Users\\spauliuk\\FILES\\Work_Archive\\_Data_Derived\\IO_models\\EXIOBASE2_Mon_48R_2017_1_17_ITC.mat' 
MRIO_L  = scipy.io.loadmat(Filestring_Matlab_in)['DES_WIO_L_ITC'] # The Leontief inverse
MRIO_S  = scipy.io.loadmat(Filestring_Matlab_in)['DES_WIO_S_ITC'] # The stressor matrix
MRIO_Y  = scipy.io.loadmat(Filestring_Matlab_in)['DES_WIO_Y']     # The final demand matrix
MRIO_Re = scipy.io.loadmat(Filestring_Matlab_in)['DES_WIO_RegionList']    # List of regions

Let's first have a look at the shape of the arrays:

In [14]:
print(MRIO_Y.shape)
print(MRIO_L.shape)
print(MRIO_S.shape)

(7824, 288)
(7824, 7824)
(692, 7824)


## Let us have a look at the Y vector:
Y is not a vector but a matrix, where for each of the 48 regions the following 6 final demand groups are recorded:
+ Final consumption expenditure by households
+ Final consumption expenditure by non-profit organisations serving households (NPISH)
+ Final consumption expenditure by government
+ Gross fixed capital formation
+ Changes in inventories
+ Changes in valuables

For national footprint calculations, the latter two columns are usually ignored, as they contain goods that do not provide service during the accounting period. We therefore slice the Y vector.

For example, to obtain the final demand of Germany, we specify:

In [17]:
Y_DE = MRIO_Y[:,30:34]

This is because Germany is no. 6 in the country list, so there are five other countries with 6 final demand columns each coming before Germany, and occupying columns 0 to 29. The slicing parameter 30:34 selects columns 30, 31, 32, and 33. 

The first column of Y_DE is the column for household final demand. This column contains the final demand of all German households, for all products in from all regions. For example, final demand for product no. 90, motor vehicles, is fullfilled both with domestic production and with imports from other regions:

In [36]:
Y_DE_HH_MotorVehicles = Y_DE[90::163,0]
ind = Y_DE_HH_MotorVehicles.argsort() # sort

for m in range(0,48):
    print(MRIO_Re[ind[47-m]], ' ', Y_DE_HH_MotorVehicles[ind[47-m]], 'MEUR')

Germany                41143.7130859 MEUR
France                 2842.31938362 MEUR
Italy                  1960.87597672 MEUR
Austria                1853.33673309 MEUR
Spain                  1806.32841972 MEUR
Hungary                1723.41290664 MEUR
Czech Republic         1721.1626944 MEUR
United States          1545.8108521 MEUR
United Kingdom         1426.4290719 MEUR
Belgium                1288.56005124 MEUR
Poland                 1250.35164902 MEUR
Slovakia               914.730055012 MEUR
Japan                  817.205253288 MEUR
Netherlands            614.778937028 MEUR
Brazil                 567.803261993 MEUR
Mexico                 540.651379438 MEUR
Turkey                 476.816409415 MEUR
South Korea            476.169766556 MEUR
Sweden                 455.40406584 MEUR
Portugal               303.097152362 MEUR
Finland                238.084275915 MEUR
Slovenia               186.400073126 MEUR
South Africa           164.248718803 MEUR
Romania                159.360433979 M

## First MRIO analyses with the loaded files:
1) Obtain the entire global emissions

In [11]:
print('Tbd.')

Tbd.
